In [1]:
#!/usr/bin/env python
# import analyzer
import importlib
from importlib import reload
import os, sys, glob, warnings, glob
import scipy
import numpy as np
import scipy as sp
import joblib
# from tqdm.notebook import tqdm
from tqdm import tqdm
import copy as cp

# ROOT
import ROOT as root

# Matplotlib
import matplotlib.pyplot as plt
from matplotlib import collections, colors, transforms

from pylab import *
%matplotlib inline
%config InlineBackend.figure_format='retina'

Welcome to JupyROOT 6.26/14


In [2]:
# Using all files, rather than just one 
# data_top_dir = f"/home/owhgabri/My_GitHub/data/DigiOutput/1xCosmicBackground"
data_top_dir = f"/home/owhgabri/My_GitHub/data/DigiOutput/LLP_bb"
pathList=[]

for rootFile, dirs, files in os.walk(data_top_dir):
    for filename in files:
        if "stat0" in filename:
            pathList.append(os.path.join(rootFile, filename))

dataFile = f"/home/owhgabri/My_GitHub/data/DigiOutput/0xCosmicBackground/24/0/stat0.root"

tfile = root.TFile.Open(dataFile, "READ")
Tree = tfile.Get("integral_tree")

branches = Tree.GetListOfBranches()
nevents = int(Tree.GetEntries())
branch_list = [branches[i].GetName() for i in range(len(branches))]

print(nevents)
print(branch_list)

347
['NumHits', 'Hit_energy', 'Hit_time', 'Hit_Center1', 'Hit_Center2', 'Hit_bar_direction', 'Hit_layer_direction', 'Hit_LayerID', 'Hit_particlePdgId', 'Hit_G4TrackId', 'Hit_G4ParentTrackId', 'Hit_x', 'Hit_y', 'Hit_z', 'Hit_particleEnergy', 'Hit_particlePx', 'Hit_particlePy', 'Hit_particlePz', 'Hit_type', 'Digi_numHits', 'Digi_time', 'Digi_x', 'Digi_y', 'Digi_z', 'Digi_energy', 'Digi_px', 'Digi_py', 'Digi_pz', 'Digi_particle_energy', 'Digi_pdg_id', 'Digi_track_id', 'Digi_type', 'Digi_center1', 'Digi_center2', 'Digi_bar_direction', 'Digi_layer_direction', 'Digi_layer_id', 'Digi_det_id', 'Digi_seed', 'Digi_hitIndices', 'GenParticle_index', 'GenParticle_G4index', 'GenParticle_pdgid', 'GenParticle_time', 'GenParticle_x', 'GenParticle_y', 'GenParticle_z', 'GenParticle_energy', 'GenParticle_px', 'GenParticle_py', 'GenParticle_pz', 'GenParticle_mass']


In [3]:
nPossible = 0
nTotalMuons = 0
for f in pathList:
    tfile = root.TFile.Open(f, "READ")
    Tree = tfile.Get("integral_tree")
    nevents = int(Tree.GetEntries())
    for i in range(nevents):
        Tree.GetEntry(i)
        curr = 0
        tracks_used = []
        for j in range(len(Tree.Digi_track_id)):
            if (Tree.Digi_track_id[j] not in tracks_used) and (Tree.Digi_layer_id[j] >=4 and Tree.Digi_type[j] != 2):
                trackId = Tree.Digi_track_id[j]
                tracks_used.append(trackId)
                if abs(Tree.Digi_pdg_id[j]) == 13:
                    nTotalMuons +=1
                layers_used = [Tree.Digi_layer_id]
                for k in range(len(Tree.Digi_track_id)):
                    if Tree.Digi_layer_id[k] not in layers_used and Tree.Digi_layer_id[k] >=4 and Tree.Digi_track_id[k] == trackId:
                    # if Tree.Digi_layer_id[k] not in layers_used and Tree.Digi_layer_id[k] >=4 and Tree.Digi_layer_id[k] <= 7:
                        layers_used.append(Tree.Digi_layer_id[k])
                if len(layers_used) >=3:
                    nPossible += 1
print("Total number of muons with hits in sensitive detector:", nTotalMuons)
print("Maximum number of possible tracks:", nPossible)
                    

Total number of muons with hits in sensitive detector: 142552
Maximum number of possible tracks: 757361
